# pHcalc *vs* phreeqpython: NaOH solutions in equilibrium with atmospheric CO2

*M. H. V. Werts, 2023*

## Introduction

`pHcalc` uses simple mass-action law with user-provided constant pKa's and Kw. It does not take into account the effects of ionic strength on chemical activity, temperature dependence etc., working best at high dilution. There is no database of property data. All this is the domain of PHREEQC (https://www.usgs.gov/software/phreeqc-version-3), a more complex piece of software that has been developed by many scientists (mainly geochemists) over many years.

There is a very good, mostly 'stand-alone', Python interface to PHREEQC called `phreeqpython` (https://github.com/Vitens/phreeqpython) which can be imported as a module into Python. It is very powerful software, able to take into account the many coupled equilibria involving many chemical species found in the natural environment. In being powerful, it may be too complicated for simple, occasional, 'quick & dirty' usage in the estimation of pH of simple aqueous solutions. That is more the domain of `pHcalc`.

Here, we compare the results by `phreeqpython` and `pHcalc` for a benchmark case. We calculate the pH of various aqueous NaOH solutions in equilibrium with atmospheric CO2, *i.e.* the pH that is obtained after prolonged bubbling of ambient air through a freshly prepared solution of NaOH in water. 


## phreeqpython usage notes

To calculate the solution composition in equilibrium with atmospheric CO2, the method `equalize` (of the `Solution` object) is used. Looking into the phreeqpython source code, the `equalize` method uses PHREEQC's `EQUILIBRIUM_PHASES` keyword.

https://water.usgs.gov/water-resources/software/PHREEQC/documentation/phreeqc3-html/phreeqc3-13.htm

Indeed, the gas partial pressures are given by their log10 value:

"saturation index --Target saturation index for the pure phase in the aqueous phase (line 1a); **for gases, this number is the log of the partial pressure (line 1b).** The target saturation index (partial pressure) may not be attained if the amount of the phase in the assemblage is insufficient. Default is 0.0."

We use the Pitzer database, which apparently is the 'best' dataset. According to PHREEQC v3 documentation (p. 8): "One limitation of the aqueous model is lack of internal consistency in the data in the databases. The database `pitzer.dat` defines the most consistent aqueous model; however, it includes only a limited number of elements.")

## Other notes

`aqion` refers to results obtained via an on-line pH calculator (which uses PHREEQC as a back-end), mainly to check if we used `phreeqpython` correctly.

DIC means 'dissolved inorganic carbon' (https://en.wikipedia.org/wiki/Dissolved_inorganic_carbon)

Of course, the `phreeqpython` module should be available somewhere in your Python installation path, if you want to run this Notebook.

## Concluding remarks

This very small study is interesting since it shows a concrete comparison of the results obtained using a simple model (mass action law with constant coefficients) to those from a highly developed model based on many experimental observations. Even for 0.1 M NaOH, the simple model is still accurate to within 0.3 pH units (lucky for us, pH is a logarithmic scale!).

In [1]:
import sys
sys.path.append('../../phreeqpython-master')

In [2]:
import numpy as np
from phreeqpython import PhreeqPython

In [3]:
from example_air_equilibrated_AcidGasEq import example2_benchmark as pHcalc_benchmark

In [4]:
pp = PhreeqPython(database = 'pitzer.dat')

In [5]:
# CO2 value used by aqion (watch out: aqion uses pCO2, not partial pressure)
aqionpCO2 = -3.408

aqionP_CO2 = 10**aqionpCO2 # conversion of pCO2 into P_CO2 [atm]
print('Using partial CO2 pressure: {0:5.3e} atm (aqion value)'.\
      format(aqionP_CO2))

Using partial CO2 pressure: 3.908e-04 atm (aqion value)


In [6]:
# Test cases
#   phreeqpython and pHcalc use different methods of defining system, hence different paramlists

paramlist_phreeq = [
    {'descr':       '0.1 M HCl(aq) in eq. with atmosphere',
     'specstr':     'HCl',
     'conc_mM':     100.,
     'aqion_pH':    1.08},
    {'descr':       '1 mM HCl(aq) in eq. with atmosphere',
     'specstr':     'HCl',
     'conc_mM':     1.,
     'aqion_pH':    3.02},
    {'descr':       'pure water in eq. with atmosphere',
     'specstr':   None,
     'conc_mM':     None,
     'aqion_pH':    5.61},
    {'descr':       '1 mM NaOH(aq) in eq. with atmosphere',
     'specstr':     'NaOH',
     'conc_mM':     1.,
     'aqion_pH':    8.20},
    {'descr':       '10 mM NaOH(aq) in eq. with atmosphere',
     'specstr':     'NaOH',
     'conc_mM':     10.,
     'aqion_pH':    9.11},
    {'descr':       '0.1 M NaOH(aq) in eq. with atmosphere',
     'specstr':     'NaOH',
     'conc_mM':     100.,
     'aqion_pH':    9.71},
    ]

paramlist_phcalc =  [
        {'descr':       '0.1 M HCl(aq) in eq. with atmosphere',
         'ioncharge':  -1,
         'ionconc':     0.1,
         'aqion_pH':    1.08},
        {'descr':       '1 mM HCl(aq) in eq. with atmosphere',
         'ioncharge':  -1,
         'ionconc':     0.001,
         'aqion_pH':    3.02},
        {'descr':       'pure water in eq. with atmosphere',
         'ioncharge':   None,
         'ionconc':     None,
         'aqion_pH':    5.61},
        {'descr':       '1 mM NaOH(aq) in eq. with atmosphere',
         'ioncharge':   1,
         'ionconc':     0.001,
         'aqion_pH':    8.20},
        {'descr':       '10 mM NaOH(aq) in eq. with atmosphere',
         'ioncharge':   1,
         'ionconc':     0.01,
         'aqion_pH':    9.11},
        {'descr':       '0.1 M NaOH(aq) in eq. with atmosphere',
         'ioncharge':   1,
         'ionconc':     0.1,
         'aqion_pH':    9.71},
        ]

In [7]:
print('phreeqpython')
print('============')
print('Using partial CO2 pressure: {0:5.3e} atm (aqion value)'.\
      format(aqionP_CO2))
for param in paramlist_phreeq:
    if param['specstr'] is not None:
        soln = pp.add_solution_simple({param['specstr']:param['conc_mM']},
                                      temperature=25.)
    else:
        soln = pp.add_solution_simple({},
                                      temperature=25.)
    soln.equalize(['CO2(g)'], [aqionpCO2])
    DIC = soln.species_moles['CO2']+soln.species_moles['HCO3-']+soln.species_moles['CO3-2']
    print('{0:40s} | DIC = {1:7.3e} M | pH = {2:5.2f} [aqion: {3:5.2f}]'.\
          format(param['descr'], DIC, soln.pH, param['aqion_pH']))

phreeqpython
Using partial CO2 pressure: 3.908e-04 atm (aqion value)
0.1 M HCl(aq) in eq. with atmosphere     | DIC = 1.331e-05 M | pH =  1.08 [aqion:  1.08]
1 mM HCl(aq) in eq. with atmosphere      | DIC = 1.331e-05 M | pH =  3.02 [aqion:  3.02]
pure water in eq. with atmosphere        | DIC = 1.576e-05 M | pH =  5.61 [aqion:  5.61]
1 mM NaOH(aq) in eq. with atmosphere     | DIC = 1.004e-03 M | pH =  8.20 [aqion:  8.20]
10 mM NaOH(aq) in eq. with atmosphere    | DIC = 9.212e-03 M | pH =  9.13 [aqion:  9.11]
0.1 M NaOH(aq) in eq. with atmosphere    | DIC = 7.047e-02 M | pH =  9.75 [aqion:  9.71]


In [8]:
print('pHcalc')
print('======')
pHcalc_benchmark(paramlist_phcalc)

pHcalc
Using partial CO2 pressure: 3.908e-04 atm (aqion value)
0.1 M HCl(aq) in eq. with atmosphere     | DIC = 1.340e-05 M | pH =  1.00 [aqion:  1.08]
1 mM HCl(aq) in eq. with atmosphere      | DIC = 1.341e-05 M | pH =  3.00 [aqion:  3.02]
pure water in eq. with atmosphere        | DIC = 1.585e-05 M | pH =  5.61 [aqion:  5.61]
1 mM NaOH(aq) in eq. with atmosphere     | DIC = 1.004e-03 M | pH =  8.22 [aqion:  8.20]
10 mM NaOH(aq) in eq. with atmosphere    | DIC = 9.396e-03 M | pH =  9.17 [aqion:  9.11]
0.1 M NaOH(aq) in eq. with atmosphere    | DIC = 7.703e-02 M | pH =  9.96 [aqion:  9.71]
